In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
preprocess_data = pd.read_pickle('./preprocessed_alcohol_v3.pkl')

with open('../category.json') as f:
    category_json = json.load(f)
with open('../flavor.json') as f:
    flavor_json = json.load(f)

In [3]:
Glass_Dict = dict()
measure = []

with open('../glass_pre.txt', 'r') as file:
    for line in file:
        data = line.strip().split(' : ')
        Glass_Dict[data[0].lower()]=float(data[1])
print(Glass_Dict)
# print(measure)

{'beer glass': 435.0, 'beer mug': 415.0, 'beer pilsner': 350.0, 'brandy snifter': 210.0, 'champagne flute': 240.0, 'cocktail glass': 195.0, 'coffee mug': 295.0, 'collins glass': 355.0, 'copper mug': 250.0, 'cordial glass': 60.0, 'highball glass': 295.0, 'hurricane glass': 590.0, 'irish coffee cup': 250.0, 'jar': 775.0, 'margarita glass': 400.0, 'coupette glass': 230.0, 'martini glass': 192.5, 'mason jar': 500.0, 'old-fashioned glass': 240.0, 'parfait glass': 250.0, 'pint glass': 520.0, 'pitcher glass': 2050.0, 'pitcher': 2050.0, 'pousse cafe glass': 180.0, 'punch bowl': 4000.0, 'shot glass': 35.0, 'whiskey sour glass': 230.0, 'white wine glass': 225.0, 'wine glass': 150.0, 'margarita/coupette glass': 400.0}


In [2]:
preprocess_data.columns

Index(['DrinkName', 'ModifiedDate', 'DrinkID', 'Alcoholic', 'Category',
       'ImageUrlPath', 'Glass', 'IBA', 'Instruction', 'Ingredients',
       'Measures'],
      dtype='object')

In [3]:
preprocess_data.Ingredients

0                      [[creme de cacao, 1], [vodka, 2]]
1      [[absolut kurant, 1], [grand marnier, 2], [cha...
2                             [[lager, 1], [tequila, 2]]
3      [[malibu rum, 1], [light rum, 2], [151 proof r...
4      [[dark rum, 1], [light rum, 2], [vodka, 3], [o...
                             ...                        
541             [[amaretto, 1], [rum, 2], [kool-aid, 3]]
542    [[cointreau, 1], [lemon juice, 2], [ice, 3], [...
543    [[beer, 1], [root beer, 2], [lemonade, 3], [co...
544                              [[vodka, 1], [ouzo, 2]]
545    [[sambuca, 1], [bailey's irish cream, 2], [whi...
Name: Ingredients, Length: 546, dtype: object

In [7]:
Iname,order = preprocess_data.loc[0, 'Ingredients'][0]
len(preprocess_data.loc[0, 'Ingredients'])


In [4]:
glass_list = list()
for glass in preprocess_data.Glass:
    glass_list.append(Glass_Dict[glass.lower()])
preprocess_data['Glass_measure'] = glass_list

In [5]:
conversion_ml_factors = {
    'ml': 1.0,
    'oz': 29.5735,
    'cl': 10.0,
    'tsp': 4.92892,
    'tblsp': 14.7868,
    'dash':0.625,
    'cup': 236.588,
    'pinch': 0.31,
    'shot': 44.3603,
    'glasses':256.3,
    'drop': 0.05,
    'pint': 473.176,
    'can': 330.0,
    'jigger': 44.3603,
    'dl':100,
    'bottle': 750.0,
    'pint':473.176,
    'quart': 946.353,
    'qt': 946.353,
    'l': 1000.0,
    'splash': 29.5735,
    'gal':3785.41,
    'dash_of': 0.92,
    'scoop': 45.0,
    'handful': 60.0,
    'piece': 30.0
    # 'stick': 113.0,
    # 'package': 500.0,
    # 'leaves': 1.0,
    # 'sprig': 1.0,
    # 'wedge': 1.0,
    # 'twist of': 1.0,
    # 'long strip': 1.0,
    # 'whole, large': 1.0,
    # 'chunks': 1.0,
    # 'to taste, by taste': 1.0,

}

In [6]:
def convert_volume(volume, unit):

    
    if unit in conversion_ml_factors:
        return volume * conversion_ml_factors[unit]
    else:
        return None

def convert_weight(weight,unit):
    conversion_factors={
        'lb':450,
        'kg':1000,
        'package':300
    }
    if unit in conversion_factors:
        return weight * conversion_factors[unit]
    else:
        return None
def convert_juice_of(unit):
    return None

# oz_to_ml(volume) = volume * 29.5735

In [19]:
#Glass 정보로 전처리
# 확인이 필요한 재료정보 -> ml로 변경해야함
last_num = list()
for drink,ingredient,measures,glass in zip(preprocess_data.DrinkName,preprocess_data.Ingredients,preprocess_data.Measures,preprocess_data.Glass):
    #재료수
    i_num = len(ingredient)
    m_num = len(measures)
    last_num.append(i_num-1)
# print(last_num)
Garnish_List = ['slice', 'sprig', 'wedge', 'twist of', 'long strip', 'whole, large', 'piece', 'chunks', 'to taste, by taste', 'handful', 'stick', 'leaves']
for ingredient,measures,glass in zip(preprocess_data.Ingredients, preprocess_data.Measures, preprocess_data.Glass_measure):
    for i in range(len(ingredient)):
        Iname,order = ingredient[i]
        # print(f"{Iname} : {measures[i][0]}")
        R=0
        if len(measures[i][0])==1:
            #'fill','fill to top','top up with' : 다른 재료를 보고 glass에 맞춰서 처리
                #분수가 입력되었을 경우
            #'wedge' : garnish로 처리 -> garnish list에 넣기
            #'twist of' : garnish로 처리 -> garnish list에 넣기
            #'to taste','by taste' : 모아놓기
            #'garnish': garnish로 처리 -> garnish list에 넣기 
            #'a little bit of' : garnish로 처리 -> garnish list에 넣기 
            #'pinch'
            #'cubes' : ice열을 0으로 초기화 후, cubes 있는 경우에 1 기입
            

            unit = measures[i][0]
            M =0
            print(f"{unit}")
            # print(f"[1] : {Iname} : unit ={unit}")
        elif len(measures[i][0])==2:
            #Normal Case
            #part, whole, 
            M,unit = measures[i][0]
            R =M
            if('/' in M):
                a,b = M.split('/')
                R = float(a)/float(b)
            elif('-' in M):
                a,b = M.split('-')
                R = (float(a) + float(b))/2

            # print(f"[2] : {Iname} : {R} {unit}, original unit:{M} {unit}")
        elif len(measures[i][0])==3:
            M,N,L = measures[i][0]
            # num, num, unit 
            if('/' in N):
                a,b = N.split('/')
                N = float(a)/float(b)
                R = float(M) + N
            elif('/' in M):
                a,b = M.split('/')
                M = float(a)/float(b)
                R = float(M) + float(N)
            else:
                print(f"{Iname} : {M} {N} {L}")
            # print(f"[3] : {Iname} : {R} {L}, original unit:{M} {N} {L}")
            # num, unit, desc
        else:
            print(f"Length Error ! : {Iname} : {measures[i][0]}")


        # if Iname in Garnish_List:
            # elif('juice of'==unit):

            #     print(f"{Iname} : {M} {unit}")
            # volume = convert_volume(float(M),unit)

        # if Iname in Garnish_List:
        #     print(f"{Iname} : {volume} Garnish")


# for num, ingredient,measures,glass in zip(last_num,preprocess_data.Ingredients, preprocess_data.Measures, preprocess_data.Glass_measure):
    # print(f"{ingredient[num]} : {measures[num][0]}")
    # print()
    # print(glass)
    
    # 단위를 통일해보자
    # print(f"{drink} : {ingredient}")

# for i in range(len(preprocess_data.loc[0, 'Ingredients'])):
#     Iname,order = preprocess_data.loc[0, 'Ingredients'][i]
    
#     print(f"{Iname}_{order} : {category_json[Iname][0]}")

# category_json['grand marnier'][0]

['fill']
['cubes']
['fill']
['cubes']
['fill']
['fill']
['1/3']
['1/3']
['1/3']
['cubes']
['top up with']
['2/3']
['1/3']
['fill']
['top up with']
['cubes']
['top up with']
['fill to top']
['by taste']
['cubes']
['by taste']
['1/2']
['1/2']
['top up with']
['fill']
['top up with']
['1/2']
['1/4']
['top up with']
['fill to top']
['top up with']
['twist of']
['a little bit of']
['fill to top']
['top up with']
['top up with']
['twist of']
['twist of']
['cubes']
['cubes']
['fill']
['cubes']
['to taste']
['cubes']
['fill']
['fill']
['top up with']
['1/3']
['1/3']
['1/4']
['top up with']
['garnish']
['to taste']
['top up with']
['fill']
['twist of']
['fill']
['twist of']
['cubes']
['top up with']
['1/2']
['1/2']
['1/2']
['to fill']
['fill']
['a little bit of']
['top up with']
['twist of']
['a little bit of']
['garnish']
['garnish']
['1/3']
['1/3']
['1/3']
['1/4']
['1/4']
['1/4']
['1/4']
['fill']
['fill']
['cubes']
['fill to top']
['fill']
['garnish']
['fill']
['fill']
['top up with']
['garni